In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms, datasets
from torchvision.models import resnet50, ResNet50_Weights
from torch.optim import Adam
from tqdm.auto import tqdm
from timeit import default_timer as timer

c:\Users\USER\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [7]:
data_path = r"C:\Users\USER\Desktop\rice leaf diseases dataset"

In [8]:
classes = [dir for dir in os.listdir(data_path)]
print(classes)

['Bacterialblight', 'Brownspot', 'Leafsmut']


In [11]:
import splitfolders
splitfolders.ratio(r"C:\Users\USER\Desktop\rice leaf diseases dataset", output="splitted", seed=1337, ratio=(.8, 0,0.2)) 

Copying files: 4684 files [01:14, 62.66 files/s] 


In [12]:
test_data_path = r"C:\Users\USER\Desktop\Rice\splitted\test"
train_data_path = r"C:\Users\USER\Desktop\Rice\splitted\train"
classes = [dir for dir in os.listdir(test_data_path)]

In [14]:
train_transform_rn50 = transforms.Compose([
    transforms.Resize(size=232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(size=224),
    transforms.RandomRotation(45), # augmentation: random rotation 45 degree
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform_rn50 = transforms.Compose([
    transforms.Resize(size=232, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [15]:
train_dataset_rn50 = datasets.ImageFolder(root=train_data_path, transform=train_transform_rn50)
test_dataset_rn50 = datasets.ImageFolder(root=test_data_path, transform=test_transform_rn50)

In [16]:
test_data_loader = DataLoader(test_dataset_rn50, batch_size=64, shuffle=True)
train_data_loader = DataLoader(train_dataset_rn50, batch_size=64, shuffle=True)

In [17]:
rn50_model = resnet50(weights=ResNet50_Weights.DEFAULT)
rn50_model.fc = nn.Sequential(nn.Linear(in_features=rn50_model.fc.in_features, out_features=38))
for param in rn50_model.parameters():
    param.requires_grad = False
    
for param in rn50_model.fc.parameters():
    param.requires_grad = True

rn50_model = rn50_model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\USER/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [02:18<00:00, 741kB/s] 


In [19]:
def training(model, dataloader, loss_fn, optimizer):
    model.train()
    train_loss = 0
    train_acc = 0
    
    for batch, (image, label) in enumerate(dataloader):
        image = image.to(device)
        label = label.to(device)
        
        label_pred = model(image)
        
        loss = loss_fn(label_pred,label)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        label_pred_class = torch.argmax(torch.softmax(label_pred, dim=1), dim=1)
        train_acc += (label_pred_class == label).sum().item()/len(label_pred)
        
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [20]:
loss_fn = nn.CrossEntropyLoss()
optim_rn50 = Adam(params=rn50_model.parameters(), lr=0.001)

In [21]:
def train_process(model,train_dataloader,optimizer,loss_fn,epochs):

    results = {"train_loss": [],"train_acc": []}

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = training(model=model,dataloader=train_dataloader,loss_fn=loss_fn,optimizer=optimizer)
        print(f"Epoch: {epoch+1} | " f"train_loss: {train_loss:.4f} | " f"train_acc: {train_acc:.4f} | ")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
    return results

In [23]:
import time
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 10

start_time = timer()

rn50_results = train_process(model=rn50_model,train_dataloader=train_data_loader,optimizer=optim_rn50,loss_fn=loss_fn,epochs=NUM_EPOCHS)

end_time = timer()
print(f"Total Training Time: {end_time-start_time:.3f} seconds")

 10%|█         | 1/10 [13:41<2:03:16, 821.87s/it]

Epoch: 1 | train_loss: 0.3165 | train_acc: 0.9249 | 


 20%|██        | 2/10 [24:06<1:34:07, 705.91s/it]

Epoch: 2 | train_loss: 0.2604 | train_acc: 0.9369 | 


 30%|███       | 3/10 [36:39<1:24:50, 727.19s/it]

...

Epoch: 8 | train_loss: 0.1474 | train_acc: 0.9722 | 


 90%|█████████ | 9/10 [1:38:36<10:22, 622.87s/it]

Epoch: 9 | train_loss: 0.1391 | train_acc: 0.9689 | 


100%|██████████| 10/10 [1:48:43<00:00, 652.36s/it]

Epoch: 10 | train_loss: 0.1286 | train_acc: 0.9710 | 
Total Training Time: 6523.607 seconds


In [24]:
def test_step(model,dataloader,loss_fn):
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            val_pred_logits = model(X)
            loss = loss_fn(val_pred_logits, y)
            test_loss += loss.item()
            val_pred_labels = val_pred_logits.argmax(dim=1)
            test_acc += ((val_pred_labels == y).sum().item()/len(val_pred_labels))
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [25]:
test_step(model=rn50_model, dataloader=test_data_loader, loss_fn=loss_fn)

(0.16251679360866547, 0.9669969512195122)